# Getting Started with PyFixest

In a first step, we load the module and some example data:

In [1]:
%load_ext autoreload
%autoreload 2

from pyfixest.estimation import feols, fepois
from pyfixest.summarize import summary
from pyfixest.visualize import coefplot, iplot
from pyfixest.utils import get_data

In [2]:
data = get_data()
data.head()

,Y,Y2,X1,X2,f1,f2,f3,group_id,Z1,Z2
0,NaN,-9.902164,0.0,0.457858,9.0,15.0,0.0,1.0,0.026577,0.872533
1,2.486016,-3.571090,NaN,-4.998406,8.0,6.0,9.0,11.0,NaN,-4.233272
2,1.449755,-3.721375,1.0,1.558480,NaN,11.0,0.0,18.0,0.049540,2.803328
3,5.179868,14.696121,2.0,1.560402,15.0,1.0,4.0,15.0,0.517344,2.151939
4,0.825537,-6.936620,1.0,-3.472232,20.0,19.0,9.0,5.0,-0.708624,-3.526701


## OLS Estimation

We can estimate a fixed effects regression via the `feols()` function. `feols()` has three arguments: a two-sided model formula, the data, and optionally, the type of inference.

In [3]:
fit = feols(fml="Y~X1 | f1", data=data, vcov="HC1")
type(fit)

pyfixest.feols.Feols

The first part of the formula contains the dependent variable and "regular" covariates, while the second part contains fixed effects.

`feols()` returns an instance of the `Fixest` class.

To inspect the results, we can use a summary function or method:

In [4]:
fit.summary()

###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  HC1
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.373 |        0.056 |     6.694 |      0.000 |   0.264 |    0.482 |
---
RMSE: 1.422  Adj. R2: 0.041  Adj. R2 Within: 0.041


Alternatively, the `.summarize` module contains a `summary` function, which can be applied on instances of regression model objects 
or lists of regression model objects. 

In [5]:
summary(fit)

###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  HC1
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.373 |        0.056 |     6.694 |      0.000 |   0.264 |    0.482 |
---
RMSE: 1.422  Adj. R2: 0.041  Adj. R2 Within: 0.041


You can access individual elements of the summary via dedicated methods: `.tidy()` returns a "tidy" `pd.DataFrame`, 
`.coef()` returns estimated parameters, and `se()` estimated standard errors. Other methods include `pvalue()`, `confint()`
and `tstat()`.

In [6]:
fit.coef()

Coefficient
X1    0.373086
Name: Estimate, dtype: float64

In [7]:
fit.se()

Coefficient
X1    0.055733
Name: Std. Error, dtype: float64

## Standard Errors and Inference

Supported covariance types are "iid", "HC1-3", CRV1 and CRV3 (one-way clustering). Inference can be adjusted "on-the-fly" via the
`.vcov()` method:

In [8]:
fit.vcov({"CRV1": "group_id"}).summary()
fit.vcov({"CRV3": "group_id"}).summary()

###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  CRV1
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.373 |        0.063 |     5.947 |      0.000 |   0.241 |    0.505 |
---
RMSE: 1.422  Adj. R2: 0.041  Adj. R2 Within: 0.041
###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  CRV3
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.373 |        0.067 |     5.565 |      0.000 |   0.232 |    0.514 |
---
RMSE: 1.422  Adj. R2: 0.041  Adj. R2 Within: 0.041


It is also possible to run a wild (cluster) bootstrap after estimation (via the [wildboottest module](https://github.com/s3alfisc/wildboottest)):

In [9]:
fit2 = feols(fml="Y~ X1", data=data, vcov={"CRV1": "group_id"})
fit2.wildboottest(param="X1", B=999)

param                X1
t value           4.934
Pr(>|t|)            0.0
bootstrap_type       11
impose_null        True
dtype: object

Note that the wild bootstrap currently does not support fixed effects in the regression model. Supporting fixed effects is work in progress.

## IV Estimation 

It is also possible to estimate instrumental variable models with *one* endogenous variable and (potentially multiple) instruments:

In [10]:
iv_fit = feols(fml="Y2~ 1 | f1 + f2 | X1 ~ Z1 + Z2", data=data)
iv_fit.summary()

###

Estimation:  IV
Dep. var.: Y2, Fixed effects: f1+f2
Inference:  CRV1
Observations:  998

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.467 |        0.251 |     1.859 |      0.073 |  -0.047 |    0.982 |
---


If the model does not contain any fixed effects, just drop the second part of the formula above:

In [11]:
feols(fml="Y~ 1 | X1 ~ Z1 + Z2", data=data).summary()

###

Estimation:  IV
Dep. var.: Y
Inference:  iid
Observations:  998

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| Intercept     |      2.166 |        0.123 |    17.673 |      0.000 |   1.925 |    2.406 |
| X1            |      0.398 |        0.110 |     3.606 |      0.000 |   0.182 |    0.615 |
---


IV estimation with multiple endogenous variables and multiple estimation syntax is currently not supported. The syntax is "depvar ~ exog.vars | fixef effects | endog.vars ~ instruments".

## Poisson Regression 

With version `0.8.4`, it is possible to estimate Poisson Regressions (not yet on PyPi): 

In [12]:
from pyfixest.utils import get_data

pois_data = get_data(model="Fepois")
pois_fit = fepois(fml="Y~X1 | f1+f2", data=pois_data, vcov={"CRV1": "group_id"})
pois_fit.summary()

###

Estimation:  Poisson
Dep. var.: Y, Fixed effects: f1+f2
Inference:  CRV1
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |     -0.034 |        0.033 |    -1.002 |      0.316 |  -0.099 |    0.032 |
---
Deviance: 1067.364


## Multiple Estimation 

`PyFixest` supports a range of multiple estimation functionality: `sw`, `sw0`, `csw`, `csw0`, and multiple dependent variables. If multiple regression syntax is used, 
`feols()` and `fepois` returns an instance of a `FixestMulti` object, which essentially consists of a dicionary of `Fepois` or `Feols` instances.

In [13]:
multi_fit = feols(fml="Y~X1 | csw0(f1, f2)", data=data, vcov="HC1")
multi_fit

In [14]:
multi_fit.summary()

###

Estimation:  OLS
Dep. var.: Y
Inference:  HC1
Observations:  998

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| Intercept     |      2.228 |        0.088 |    25.418 |      0.000 |   2.056 |    2.400 |
| X1            |      0.336 |        0.068 |     4.932 |      0.000 |   0.202 |    0.469 |
---
RMSE: 1.765  Adj. R2: 0.021  Adj. R2 Within: 0.021
###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  HC1
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.373 |        0.056 |     6.694 |      0.000 |   0.264 |    0.482 |
---
RMSE: 1.422  Adj. R2: 0.041  Adj. R2 Within: 0.041
###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1+f2
Inference:  HC1
Observations

Alternatively, you can look at the estimation results via the `etable()` method:

In [15]:
multi_fit.etable()

fml              Y~X1        Y~X1|f1 Y~X1|f1+f2
Coefficient Intercept     X1      X1         X1
Estimate        2.228  0.336   0.373      0.402
Std. Error      0.088  0.068   0.056      0.049
t value        25.418  4.932   6.694      8.290
Pr(>|t|)        0.000  0.000   0.000      0.000
2.5 %           2.056  0.202   0.264      0.307
97.5 %          2.400  0.469   0.482      0.498

If you are only insterested in some parameters, e.g. "X1", you can use the following syntax:

In [16]:
multi_fit.etable().xs("X1", level=1, axis=1)

fml,Y~X1,Y~X1|f1,Y~X1|f1+f2
Estimate,0.336,0.373,0.402
Std. Error,0.068,0.056,0.049
t value,4.932,6.694,8.290
Pr(>|t|),0.000,0.000,0.000
2.5 %,0.202,0.264,0.307
97.5 %,0.469,0.482,0.498


You can access an individual model by its name - i.e. a formula - via the `all_fitted_models` attribure.

In [17]:
multi_fit.all_fitted_models["Y~X1"].tidy()

,Estimate,Std. Error,t value,Pr(>|t|),2.5 %,97.5 %
Coefficient,,,,,,
Intercept,2.227929,0.087650,25.418482,0.000000e+00,2.055930,2.399929
X1,0.335588,0.068044,4.931907,9.536936e-07,0.202061,0.469114


or equivalently via the `fetch_model` method:

In [18]:
multi_fit.fetch_model(0).tidy()

Model:  Y~X1


,Estimate,Std. Error,t value,Pr(>|t|),2.5 %,97.5 %
Coefficient,,,,,,
Intercept,2.227929,0.087650,25.418482,0.000000e+00,2.055930,2.399929
X1,0.335588,0.068044,4.931907,9.536936e-07,0.202061,0.469114


Here, `0` simply fetches the first model stored in the `all_fitted_models` dictionary, `1` the second etc.

Objects of type `Fixest` come with a range of additional methods: `tidy()`, `coef()`, `vcov()` etc, which 
essentially loop over the equivalent methods of all fitted models. E.g. `Fixest.vcov()` updates inference for all 
models stored in `Fixest`.

In [19]:
multi_fit.vcov("iid").summary()

###

Estimation:  OLS
Dep. var.: Y
Inference:  iid
Observations:  998

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| Intercept     |      2.228 |        0.088 |    25.322 |      0.000 |   2.055 |    2.401 |
| X1            |      0.336 |        0.069 |     4.876 |      0.000 |   0.201 |    0.471 |
---
RMSE: 1.765  Adj. R2: 0.021  Adj. R2 Within: 0.021
###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  iid
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.373 |        0.057 |     6.592 |      0.000 |   0.262 |    0.484 |
---
RMSE: 1.422  Adj. R2: 0.041  Adj. R2 Within: 0.041
###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1+f2
Inference:  iid
Observations

## Visualization 

`PyFixest` provides two functions to visualize the results of a regression: `coefplot` and `iplot`.

In [20]:
from lets_plot import *

LetsPlot.setup_html()

multi_fit.coefplot().show()

## TWFE Event Study

To conclude this intro, we estimate an event study from an example of the the [LOST](https://lost-stats.github.io/Model_Estimation/Research_Design/event_study.html) library of statistical techniques.

In [21]:
import pandas as pd
import numpy as np

# Read in data
df = pd.read_csv(
    "https://raw.githubusercontent.com/LOST-STATS/LOST-STATS.github.io/master/Model_Estimation/Data/Event_Study_DiD/bacon_example.csv"
)

df["time_to_treat"] = (df["year"] - df["_nfd"]).fillna(0).astype(int)
df["time_to_treat"] = pd.Categorical(
    df.time_to_treat, np.sort(df.time_to_treat.unique())
)
df["treat"] = np.where(pd.isna(df["_nfd"]), 0, 1)

fml = "asmrs ~ i(time_to_treat, treat, ref = -1) + csw(pcinc, asmrh, cases) | stfips + year"
fit = feols(fml=fml, data=df, vcov={"CRV1": "stfips"})

In [22]:
plot = fit.iplot(
    yintercept=0,
    figsize=(2000, 600),
    coord_flip=False,
    title="Event Study Plot",
    rotate_xticks=90,
)
plot.show()